<a href="https://colab.research.google.com/github/mohansameer1983/ComputerVisionProjects/blob/main/Trash_Classification_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import cv2
from google.colab.patches import cv2_imshow      # To show images using cv2 module

import pathlib

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Input, Dropout
from keras.preprocessing.image import ImageDataGenerator

import tensorflow_hub as hub

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TF version: 2.8.0
Hub version: 0.12.0
GPU is NOT AVAILABLE


# Model Selection

In [2]:
  # Select base model  for transfer learning
  model_name = "mobilenet_v3_small_100_224" # @param [ 'resnet_v1_50', 'resnet_v1_101', 'resnet_v1_152', 'resnet_v2_50', 'resnet_v2_101', 'resnet_v2_152',  'mobilenet_v2_100_224', 'mobilenet_v2_130_224', 'mobilenet_v2_140_224', 'mobilenet_v3_small_100_224', 'mobilenet_v3_small_075_224', 'mobilenet_v3_large_100_224', 'mobilenet_v3_large_075_224']

  model_handle_map = {
    "resnet_v1_50": "https://tfhub.dev/google/imagenet/resnet_v1_50/feature-vector/4",
    "resnet_v1_101": "https://tfhub.dev/google/imagenet/resnet_v1_101/feature-vector/4",
    "resnet_v1_152": "https://tfhub.dev/google/imagenet/resnet_v1_152/feature-vector/4",
    "resnet_v2_50": "https://tfhub.dev/google/imagenet/resnet_v2_50/feature-vector/4",
    "resnet_v2_101": "https://tfhub.dev/google/imagenet/resnet_v2_101/feature-vector/4",
    "resnet_v2_152": "https://tfhub.dev/google/imagenet/resnet_v2_152/feature-vector/4",
    "nasnet_large": "https://tfhub.dev/google/imagenet/nasnet_large/feature_vector/4",
    "nasnet_mobile": "https://tfhub.dev/google/imagenet/nasnet_mobile/feature_vector/4",
    "pnasnet_large": "https://tfhub.dev/google/imagenet/pnasnet_large/feature_vector/4",
    "mobilenet_v2_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4",
    "mobilenet_v2_130_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/feature_vector/4",
    "mobilenet_v2_140_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4",
    "mobilenet_v3_small_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5",
    "mobilenet_v3_small_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/feature_vector/5",
    "mobilenet_v3_large_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5",
    "mobilenet_v3_large_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5",
  }

  PIXELS = 224#@param {type:"integer"}
  BATCH_SIZE = 16#@param {type:"integer"}

  model_handle = model_handle_map.get(model_name)

  print(f"Selected model: {model_name} : {model_handle}")

  IMAGE_SIZE = (PIXELS, PIXELS)
  print(f"Input size {IMAGE_SIZE}")


Selected model: mobilenet_v3_small_100_224 : https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5
Input size (224, 224)


# Data Preparation

In [3]:
# Mount Google drive so dataset can be accessed
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
def load_data(save_dir="./"):
  data_dir = pathlib.Path('/content/drive/MyDrive/Colab Notebooks/data/trash') 

  train_datagen = ImageDataGenerator(
          rescale=1./255,
          horizontal_flip=True)

  train_generator = train_datagen.flow_from_directory(
          data_dir,
          target_size=(PIXELS, PIXELS),
          batch_size=8,
          class_mode='categorical',
          interpolation='nearest')
  return 

load_data()

Found 310 images belonging to 7 classes.


In [ ]:
def clean_data(df):

  
  return df

print(train)
train = clean_data(train)
val = clean_data(val)
test = clean_data(test)

print(train.head())
print(test.head())

       idx                                           sentence label
0     1680  b'It is this hat that it is certain that he wa...     1
1     1456  b'Her efficient looking up of the answer pleas...     1
2     4223          b'Both the workers will wear carnations.'     1
3     4093  b'John enjoyed drawing trees for his syntax ho...     1
4     7111  b'We consider Leslie rather foolish, and Lou a...     1
...    ...                                                ...   ...
8546  4920              b'a pencil with that to write broke.'     0
8547  3619  b'It was in the park last night that the polic...     0
8548  3624       b'John looked up the inside of the chimney.'     1
8549  8351                     b'You said she liked yourself'     0
8550  5073    b"What we're using is their teaching material."     1

[8551 rows x 3 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


    idx                                           sentence label
0  1680  b'it this hat that it certain that he was wear...     1
1  1456  b'her efficient looking up of answer pleased b...     1
2  4223                    b'both workers wear carnations'     1
3  4093  b'john enjoyed drawing trees for his syntax ho...     1
4  7111  b'we consider leslie rather foolish lou comple...     1
    idx                                       sentence label
0   163               b'brian was wiping behind stove'    -1
1   131          b'you could give headache to tylenol'    -1
2  1021                         b'i want to meet at 6'    -1
3   166                       b'packages carry easily'    -1
4  1039  b'many people said they were sick who werent'    -1


In [ ]:
def extract_text_and_y(df):
  text = [x.decode('utf-8') for x in  df.sentence.values]
  # for multiclass problems, you can use sklearn.preprocessing.OneHotEncoder, but we only have two classes, so we'll use a single sigmoid output
  y = np.array([x for x in df.label.values])
  return text, y

# the following prepares the input for running
train_text, train_y = extract_text_and_y(clean_data(train))
val_text, val_y = extract_text_and_y(clean_data(val))
test_text, test_y = extract_text_and_y(clean_data(test))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Modelling

## Build and Train Model


In [ ]:
def build_model(base_model, trainable=False, params={}):
   
    base_model.trainable = trainable


    # Any additional layers should go here
    # use the 'params' as a dictionary for hyper parameter to facilitate experimentation
    dense_layer = Dense(params['dense_layer_1_units'],activation='relu')(dbert_last_hidden_state[:, 0, :])
    dropout_layer = Dropout(params['dropout_rate_1'])(dense_layer)
    dense_layer = Dense(params['dense_layer_2_units'],activation='relu')(dropout_layer)
    #dropout_layer = Dropout(params['dropout_rate_2'])(dense_layer)
    probs = Dense(1, activation='sigmoid',kernel_regularizer=regularizers.l2(params['regularizer_l2_rate']), name="output")(dense_layer)

    model = keras.Model(inputs=[inputs, masks], outputs=probs)
    model.summary()
    return model

dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
params={# add parameters here
        "dense_layer_1_units": 128,
	      "dense_layer_2_units": 64,
        "dropout_rate_1": 0.1,
        "dropout_rate_2": 0.1,
        "regularizer_l2_rate":0.01
        }

model = build_model(dbert_model, params=params)

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_1[0][0]',                
 BertModel)                     ast_hidden_state=(N               'input_2[0][0]']                
                                one, 24, 768),                                                    
                                 hidden_states=None                                           

In [ ]:
def compile_model(model):
    # TODO: compile the model, include relevant auc metrics when training
    # DO NOT CHANGE THE INPUTS OR OUTPUTS TO THIS FUNCTION

    METRICS = [
          tf.keras.metrics.BinaryAccuracy(name='accuracy'),
          tf.keras.metrics.Precision(name='precision'),
          tf.keras.metrics.Recall(name='recall')
    ]
    optimizer = keras.optimizers.Adam(learning_rate=1e-5)
    loss = keras.losses.binary_crossentropy
    model.compile(optimizer=optimizer,
              loss=loss,
              metrics=METRICS)
    return model

model = compile_model(model)

In [ ]:
import tensorflow as tf
def train_model(model, model_inputs_and_masks_train, model_inputs_and_masks_val,
    y_train, y_val, batch_size, num_epochs):
    # TODO: train the model
    # DO NOT CHANGE THE INPUTS OR OUTPUTS TO THIS FUNCTION
  
    history = model.fit(dataset,
          epochs=num_epochs,
          batch_size=batch_size, verbose=1)
    
    return model, history

model, history = train_model(model, train_model_inputs_and_masks, val_model_inputs_and_masks, train_y, val_y, batch_size=128, num_epochs=5)

Epoch 1/5
1/1 [==============================] - 352s 352s/step - loss: 0.6849 - accuracy: 0.6423 - precision: 0.7025 - recall: 0.8536
Epoch 2/5
1/1 [==============================] - 350s 350s/step - loss: 0.6822 - accuracy: 0.6487 - precision: 0.7043 - recall: 0.8639
Epoch 3/5
1/1 [==============================] - 350s 350s/step - loss: 0.6817 - accuracy: 0.6494 - precision: 0.7038 - recall: 0.8673
Epoch 4/5
1/1 [==============================] - 348s 348s/step - loss: 0.6812 - accuracy: 0.6572 - precision: 0.7040 - recall: 0.8859
Epoch 5/5
1/1 [==============================] - 352s 352s/step - loss: 0.6785 - accuracy: 0.6550 - precision: 0.7015 - recall: 0.8883
